# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [37]:
csv = "../WeatherPy/worldweather.csv"

worldweather_df = pd.read_csv(csv)

worldweather_df.head()


,Unnamed: 0,city,cloudiness,country,date,humidity,lat,long,max temp,wind speed
0,0,hilo,90,US,1578272660,58,19.71,-155.08,82.40,8.05
1,1,longyearbyen,64,NO,1578272713,59,78.22,15.63,-8.16,3.94
2,2,srednekolymsk,12,RU,1578272714,81,67.46,153.71,-10.25,3.09
3,3,ribas do rio pardo,31,BR,1578272714,61,-20.45,-53.76,75.36,6.17
4,4,vaini,75,IN,1578272714,82,15.34,74.49,68.00,4.70


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [41]:
#the gmaps configure function will not, for some reason, accept the 'g_key' variable imported above
#I have to put it in directly for some reason. 

gmaps.configure(api_key=g_key)

fig = gmaps.figure()

locations = worldweather_df[["lat", "long"]].astype(float)

humidity = worldweather_df["humidity"].astype(float)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [47]:
#desirable weather: warmer than 75, cooler than 90, humidity 75% or less, cloudiness under 50

ww_edit_df = worldweather_df
ww_edit_df.rename(columns = {'max temp': 'max_temp', 'wind speed': 'windspeed'}, inplace = True)

ww_edit_df.head()

,Unnamed: 0,city,cloudiness,country,date,humidity,lat,long,max_temp,windspeed
0,0,hilo,90,US,1578272660,58,19.71,-155.08,82.40,8.05
1,1,longyearbyen,64,NO,1578272713,59,78.22,15.63,-8.16,3.94
2,2,srednekolymsk,12,RU,1578272714,81,67.46,153.71,-10.25,3.09
3,3,ribas do rio pardo,31,BR,1578272714,61,-20.45,-53.76,75.36,6.17
4,4,vaini,75,IN,1578272714,82,15.34,74.49,68.00,4.70


In [48]:
#https://stackoverflow.com/questions/13851535/delete-rows-from-a-pandas-dataframe-based-on-a-conditional-expression-involving

for location in ww_edit_df:
    ww_edit_df = ww_edit_df.drop(ww_edit_df[ww_edit_df.max_temp <= 75].index)
    ww_edit_df = ww_edit_df.drop(ww_edit_df[ww_edit_df.max_temp >= 95].index)
    ww_edit_df = ww_edit_df.drop(ww_edit_df[ww_edit_df.humidity > 75].index)
    ww_edit_df = ww_edit_df.drop(ww_edit_df[ww_edit_df.cloudiness >= 50].index)

ww_edit_df.head()
        

,Unnamed: 0,city,cloudiness,country,date,humidity,lat,long,max_temp,windspeed
3,3,ribas do rio pardo,31,BR,1578272714,61,-20.45,-53.76,75.36,6.17
8,8,atuona,34,PF,1578272714,75,-9.80,-139.03,80.55,9.46
11,11,jawa,39,ID,1578272715,75,5.32,95.93,81.19,14.47
12,12,busselton,0,AU,1578272715,49,-33.64,115.35,77.00,3.00
13,13,bagotville,40,AU,1578272715,74,-28.98,153.42,84.20,17.22


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [54]:
hotel_df = ww_edit_df

In [55]:
hotel_df['Hotel Name'] = ''

hotel_df.head()

,Unnamed: 0,city,cloudiness,country,date,humidity,lat,long,max_temp,windspeed,Hotel Name
3,3,ribas do rio pardo,31,BR,1578272714,61,-20.45,-53.76,75.36,6.17,
8,8,atuona,34,PF,1578272714,75,-9.80,-139.03,80.55,9.46,
11,11,jawa,39,ID,1578272715,75,5.32,95.93,81.19,14.47,
12,12,busselton,0,AU,1578272715,49,-33.64,115.35,77.00,3.00,
13,13,bagotville,40,AU,1578272715,74,-28.98,153.42,84.20,17.22,


In [68]:
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row['lat']
    lng = row['long']
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    name_address = requests.get(base_url, params=params)
    
    name_address = name_address.json()

    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df.head()

Missing field/result... skipping.


,Unnamed: 0,city,cloudiness,country,date,humidity,lat,long,max_temp,windspeed,Hotel Name
3,3,ribas do rio pardo,31,BR,1578272714,61,-20.45,-53.76,75.36,6.17,Hotel Curaçau
8,8,atuona,34,PF,1578272714,75,-9.80,-139.03,80.55,9.46,Villa Enata
11,11,jawa,39,ID,1578272715,75,5.32,95.93,81.19,14.47,Ilui nisa
12,12,busselton,0,AU,1578272715,49,-33.64,115.35,77.00,3.00,Observatory Guest House
13,13,bagotville,40,AU,1578272715,74,-28.98,153.42,84.20,17.22,Broadwater Sunrise Caravan Park


In [74]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>city</dt><dd>{city}</dd>
<dt>country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "long"]]
#names = hotel_df["Hotel Name"]

In [76]:
# Add marker layer ontop of heat map

symbol_layer = gmaps.symbol_layer(locations)
#symbol_layer = gmaps.symbol_layer(locations, hover_text=names)
fig.add_layer(symbol_layer)

# Display Map

fig

Figure(layout=FigureLayout(height='420px'))